In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive'
!unzip 'pt_245.zip'

/content/drive/MyDrive
Archive:  pt_245.zip
   creating: 00_csv/abnormal/
  inflating: 00_csv/abnormal/0204010907(1512)(0002)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0003)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0004)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0005)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0006)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0007)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0008)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0009)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0010)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0011)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0012)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0013)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0014)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0015)0102.csv  
  inflating: 00_csv/abnormal/0204010907(1512)(0016)0102.cs

In [ ]:
import tqdm
import glob
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
## 'list_label' :  json 파일 경로 리스트
path_label = '/content/drive/MyDrive/01_label/abnormal'
list_label = glob.glob(path_label + '/*.json')

n = []
ab= []
for path_label in list_label:
  with open(path_label) as r:
    dict_label = json.load(r)
    tmp = path_label.split('/')
    tmp[-3] = '00_csv'
    tmp2 = tmp[-1].split('.')
    tmp2[-1] = 'csv'
    tmp[-1] = '.'.join(tmp2)
    path_csv ='/'.join(tmp)

    try: 
      df = pd.read_csv(path_csv)
      env = []
      env.append(float(df.columns[0].split(';')[-1]))
      for i in range(4):
        env.append(float(df.loc[i][0].split(';')[-1]))
      
      df_temp = pd.read_csv(path_csv)[4:].reset_index(drop=True)
      image = np.zeros((512, 640))
      for idx, row in df_temp.iterrows():
        image[idx, :] = np.array([float(x) for x in row[0].split(';')[:-1]])

      for obj in dict_label["annotations"]:
        bbox = list(obj["data"].values())
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]
        
        tmp = []
        for i in range(h):
            for j in range(w):
              if y+i >= 512 or x+j >= 640:
                continue
              tmp.append(image[y+i][x+j])
        tmp.sort(reverse=True)
        if obj["attributes"]["status"] == "danger":
          #
          tmp = tmp[0:5]
          tmp.extend(env)
          ab.append(tmp)
        else:
          tmp = tmp[0:5]
          tmp.extend(env)
          n.append(tmp)
          
    except:
      print(path_csv, 'is not here!!')

In [ ]:
print(len(n))
print(len(ab))

0
1224


In [ ]:
## 'list_label' :  json 파일 경로 리스트
path_label = '/content/drive/MyDrive/01_label/normal'
list_label = glob.glob(path_label + '/*.json')

n2 = []
ab2= []
for path_label in list_label:
  with open(path_label) as r:
    dict_label = json.load(r)
    tmp = path_label.split('/')
    tmp[-3] = '00_csv'
    tmp2 = tmp[-1].split('.')
    tmp2[-1] = 'csv'
    tmp[-1] = '.'.join(tmp2)
    path_csv ='/'.join(tmp)

    try: 
      df = pd.read_csv(path_csv)
      env = []
      env.append(float(df.columns[0].split(';')[-1]))
      for i in range(4):
        env.append(float(df.loc[i][0].split(';')[-1]))
      
      df_temp = pd.read_csv(path_csv)[4:].reset_index(drop=True)
      image = np.zeros((512, 640))
      for idx, row in df_temp.iterrows():
        image[idx, :] = np.array([float(x) for x in row[0].split(';')[:-1]])

      for obj in dict_label["annotations"]:
        bbox = list(obj["data"].values())
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]
        
        tmp = []
        for i in range(h):
            for j in range(w):
              if y+i >= 512 or x+j >= 640:
                continue
              tmp.append(image[y+i][x+j])
        tmp.sort(reverse=True)
        if obj["attributes"]["status"] == "danger":
          #
          tmp = tmp[0:5]
          tmp.extend(env)
          ab2.append(tmp)
        else:
          tmp = tmp[0:5]
          tmp.extend(env)
          n2.append(tmp)
          
    except:
      print(path_csv, 'is not here!!')

In [ ]:
print(len(n2))
print(len(ab2))

1069
0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
df_normal = pd.DataFrame(n2,columns=['1','2','3','4','5','Emissivity','ReflectedTemperature','Distance','Atmospherictemperature','RelativeHumidity'])
df_abnormal = pd.DataFrame(ab,columns=['1','2','3','4','5','Emissivity','ReflectedTemperature','Distance','Atmospherictemperature','RelativeHumidity'])

dataset = pd.concat([df_normal, df_abnormal], ignore_index=True)

tmp = []
for _ in range(len(n2)):
  tmp.append(1)
for _ in range(len(ab)):
  tmp.append(0)

dataset['target'] = tmp

In [ ]:
data =dataset[['1','2','3','4','5','Emissivity','ReflectedTemperature','Distance','Atmospherictemperature','RelativeHumidity']]
target = dataset['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=34)


In [ ]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Support Vector Machines
from sklearn.svm import SVC
models['SVC'] = SVC(C=1.0, 
                      kernel='rbf',
                      degree=3,
                      gamma='scale',
                      coef0=0.0,
                      shrinking=True,
                      probability=False,
                      tol=0.001,
                      cache_size=200,
                      class_weight=None,
                      verbose=False, 
                      max_iter=- 1, 
                      decision_function_shape='ovr', 
                      break_ties=False, random_state=None)

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy, precision, recall = {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)

In [ ]:
import pandas as pd

df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()

df_model

,Accuracy,Precision,Recall
Logistic Regression,0.705882,0.537383,0.761589
Support Vector Machines,0.705882,0.518692,0.776224
SVC,0.725490,0.481308,0.872881
Decision Trees,0.623094,0.570093,0.600985
Random Forest,0.649237,0.542056,0.648045
Naive Bayes,0.705882,0.509346,0.784173
K-Nearest Neighbor,0.679739,0.551402,0.698225


In [ ]:
ax = df_model.plot.barh()
ax.legend(
    ncol=len(models.keys()), 
    bbox_to_anchor=(0, 1), 
    loc='lower left', 
    prop={'size': 14}
)
plt.tight_layout()

In [ ]:
## 'list_label' :  json 파일 경로 리스트
path_label = '/content/drive/MyDrive/0302010728(1054)(0409)0102.json'
path_csv = '/content/drive/MyDrive/0302010728(1054)(0409)0102.csv'
test_n = []
test_ab= []

with open(path_label) as r:
  dict_label = json.load(r)

df = pd.read_csv(path_csv)
env = []
env.append(float(df.columns[0].split(';')[-1]))
for i in range(4):
  env.append(float(df.loc[i][0].split(';')[-1]))

df_temp = pd.read_csv(path_csv)[4:].reset_index(drop=True)
image = np.zeros((512, 640))
for idx, row in df_temp.iterrows():
  image[idx, :] = np.array([float(x) for x in row[0].split(';')[:-1]])

for obj in dict_label["annotations"]:
  bbox = list(obj["data"].values())
  x = bbox[0]
  y = bbox[1]
  w = bbox[2]
  h = bbox[3]
  
  tmp = []
  for i in range(h):
      for j in range(w):
        if y+i >= 512 or x+j >= 640:
          continue
        tmp.append(image[y+i][x+j])
  tmp.sort(reverse=True)
  if obj["attributes"]["status"] == "danger":
    #
    tmp = tmp[0:5]
    tmp.extend(env)
    test_ab.append(tmp)
  else:
    tmp = tmp[0:5]
    tmp.extend(env)
    test_n.append(tmp)
    


In [ ]:
test_ab

[[55.82, 55.82, 55.78, 55.75, 55.73, 0.95, 35.0, 2.0, 35.0, 60.0]]

In [ ]:
## abnormal 일때 0/ normal일때 1
for key in models.keys():
    # Make predictions
    predictions = models[key].predict([[55.82, 55.82, 55.78, 55.75, 55.73, 0.95, 35.0, 2.0, 35.0, 60.0]])
    if predictions == 0:
      print(key, 'thinks it is Abnormal!!')
    else:
      print(key, 'thinks it is Normal')

Logistic Regression thinks it is Abnormal!!
Support Vector Machines thinks it is Abnormal!!
SVC thinks it is Normal
Decision Trees thinks it is Abnormal!!
Random Forest thinks it is Abnormal!!
Naive Bayes thinks it is Normal
K-Nearest Neighbor thinks it is Abnormal!!


In [ ]:
def anomaldtect(path_label, path_csv):
  ## 'list_label' :  json 파일 경로 리스트
  my_list = ['Logistic Regression','Support Vector Machines','SVC','Decision Trees','Random Forest','Naive Bayes','K-Nearest Neighbor']
  score_dic = dict.fromkeys(my_list, 0)

  test_n = []
  with open(path_label) as r:
    dict_label = json.load(r)

  df = pd.read_csv(path_csv)
  env = []
  env.append(float(df.columns[0].split(';')[-1]))
  for i in range(4):
    env.append(float(df.loc[i][0].split(';')[-1]))

  df_temp = pd.read_csv(path_csv)[4:].reset_index(drop=True)
  image = np.zeros((512, 640))
  for idx, row in df_temp.iterrows():
    image[idx, :] = np.array([float(x) for x in row[0].split(';')[:-1]])

  for obj in dict_label["annotations"]:
    bbox = list(obj["data"].values())
    x = bbox[0]
    y = bbox[1]
    w = bbox[2]
    h = bbox[3]
    
    tmp = []
    for i in range(h):
        for j in range(w):
          if y+i >= 512 or x+j >= 640:
            continue
          tmp.append(image[y+i][x+j])
    tmp.sort(reverse=True)
    
    tmp = tmp[0:5]
    tmp.extend(env)
    test_n.append(tmp)

    
    ## abnormal 일때 0/ normal일때 1
    for key in models.keys():
        # Make predictions
        predictions = models[key].predict(test_n)
        
        if (predictions == 0 and obj["attributes"]["status"] == "danger"):
           print(key, 'is right!', obj["attributes"]["status"])
           score_dic[key] += 1
        elif (predictions == 1 and obj["attributes"]["status"] == "normal"):
           print(key, 'is right!!', obj["attributes"]["status"])
           score_dic[key] += 1
        else:
           print(key, 'is wrong')
    test_n= []
          



In [ ]:
path = '/content/drive/MyDrive/PT'
json_list = glob.glob(path+'/*.json')
for path_label in json_list:
  tmp = path_label.split('/')
  tmp2 = tmp[-1].split('.')
  tmp2[-1] = 'csv'
  tmp[-1] = '.'.join(tmp2)
  path_csv ='/'.join(tmp)
  anomaldtect(path_label,path_csv)
  print('==============')

Logistic Regression is wrong
Support Vector Machines is wrong
SVC is right!! normal
Decision Trees is wrong
Random Forest is wrong
Naive Bayes is right!! normal
K-Nearest Neighbor is wrong
Logistic Regression is wrong
Support Vector Machines is wrong
SVC is right!! normal
Decision Trees is wrong
Random Forest is wrong
Naive Bayes is right!! normal
K-Nearest Neighbor is wrong
Logistic Regression is wrong
Support Vector Machines is wrong
SVC is right!! normal
Decision Trees is wrong
Random Forest is wrong
Naive Bayes is right!! normal
K-Nearest Neighbor is wrong
Logistic Regression is wrong
Support Vector Machines is wrong
SVC is right!! normal
Decision Trees is wrong
Random Forest is wrong
Naive Bayes is right!! normal
K-Nearest Neighbor is wrong
Logistic Regression is right! danger
Support Vector Machines is right! danger
SVC is wrong
Decision Trees is right! danger
Random Forest is right! danger
Naive Bayes is wrong
K-Nearest Neighbor is right! danger
Logistic Regression is wrong
Supp

danger
Logistic Regression is right!!
danger
Support Vector Machines is right!!
danger
SVC is wrong
danger
Decision Trees is right!!
danger
Random Forest is right!!
danger
Naive Bayes is wrong
danger
K-Nearest Neighbor is right!!


In [ ]:
anomaldtect('/content/drive/MyDrive/testset/0302010728(1054)(0025)0102.json','/content/drive/MyDrive/testset/0302010728(1054)(0025)0102.csv')

danger
Logistic Regression is right!!
danger
Support Vector Machines is right!!
danger
SVC is wrong
danger
Decision Trees is right!!
danger
Random Forest is right!!
danger
Naive Bayes is wrong
danger
K-Nearest Neighbor is right!!


In [ ]:
anomaldtect('/content/drive/MyDrive/testset/0302010915(1121)(2402)0101.json','/content/drive/MyDrive/testset/0302010915(1121)(2402)0101.csv')

normal
Logistic Regression is wrong
normal
Support Vector Machines is wrong
normal
SVC is right!!
normal
Decision Trees is wrong
normal
Random Forest is wrong
normal
Naive Bayes is right!!
normal
K-Nearest Neighbor is wrong


In [ ]:
anomaldtect('/content/drive/MyDrive/testset/0302010915(1121)(2456)0101.json','/content/drive/MyDrive/testset/0302010915(1121)(2456)0101.csv')

normal
Logistic Regression is wrong
normal
Support Vector Machines is wrong
normal
SVC is right!!
normal
Decision Trees is wrong
normal
Random Forest is wrong
normal
Naive Bayes is right!!
normal
K-Nearest Neighbor is wrong


In [ ]:
anomaldtect('/content/drive/MyDrive/PT/0204010818(0206)(0001)0102.json','/content/drive/MyDrive/PT/0204010818(0206)(0001)0102.csv')

normal
[0 0 0]


ValueError: ignored

In [ ]:
cd '/content/drive/MyDrive'

!unzip 

/content/drive/MyDrive
